In [2]:
import pandas as pd 

In [3]:
df_adm = pd.read_csv("C:/Users/user/Downloads/BigData/dataset/mimic-iii-clinical-database-demo-1.4/ADMISSIONS.csv")

In [4]:
df_adm.head()


,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data
0,12258,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,SEPARATED,BLACK/AFRICAN AMERICAN,2164-10-23 16:43:00,2164-10-23 23:00:00,SEPSIS,0,1
1,12263,10011,105331,2126-08-14 22:32:00,2126-08-28 18:59:00,2126-08-28 18:59:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Private,NaN,CATHOLIC,SINGLE,UNKNOWN/NOT SPECIFIED,NaN,NaN,HEPATITIS B,1,1
2,12265,10013,165520,2125-10-04 23:36:00,2125-10-07 15:13:00,2125-10-07 15:13:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,NaN,CATHOLIC,NaN,UNKNOWN/NOT SPECIFIED,NaN,NaN,SEPSIS,1,1
3,12269,10017,199207,2149-05-26 17:19:00,2149-06-03 18:42:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,CATHOLIC,DIVORCED,WHITE,2149-05-26 12:08:00,2149-05-26 19:45:00,HUMERAL FRACTURE,0,1
4,12270,10019,177759,2163-05-14 20:43:00,2163-05-15 12:00:00,2163-05-15 12:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,NaN,CATHOLIC,DIVORCED,WHITE,NaN,NaN,ALCOHOLIC HEPATITIS,1,1


In [5]:
#calc nulls
df_adm.isnull().sum()


row_id                   0
subject_id               0
hadm_id                  0
admittime                0
dischtime                0
deathtime               89
admission_type           0
admission_location       0
discharge_location       0
insurance                0
language                48
religion                 1
marital_status          16
ethnicity                0
edregtime               37
edouttime               37
diagnosis                0
hospital_expire_flag     0
has_chartevents_data     0
dtype: int64

In [6]:

text_columns = ['language', 'religion', 'marital_status', 'ethnicity', 'diagnosis']
df_adm[text_columns] = df_adm[text_columns].fillna('UNKNOWN')


In [7]:

time_columns = ['edregtime', 'edouttime']
df_adm[time_columns] = df_adm[time_columns].fillna(pd.NaT)

In [8]:
print(df_adm.isnull().sum())

row_id                   0
subject_id               0
hadm_id                  0
admittime                0
dischtime                0
deathtime               89
admission_type           0
admission_location       0
discharge_location       0
insurance                0
language                 0
religion                 0
marital_status           0
ethnicity                0
edregtime               37
edouttime               37
diagnosis                0
hospital_expire_flag     0
has_chartevents_data     0
dtype: int64


In [9]:
#convert to datetime
time_cols = ['admittime', 'dischtime', 'deathtime', 'edregtime', 'edouttime']
for col in time_cols:
    df_adm[col] = pd.to_datetime(df_adm[col], errors='coerce')

In [10]:
#convert to category
cat_cols = ['admission_type', 'admission_location', 'discharge_location', 
            'insurance', 'language', 'religion', 'marital_status', 'ethnicity']
df_adm[cat_cols] = df_adm[cat_cols].astype('category')

In [11]:
#convert to boolean 
df_adm['hospital_expire_flag'] = df_adm['hospital_expire_flag'].astype(bool)
df_adm['has_chartevents_data'] = df_adm['has_chartevents_data'].astype(bool)

In [13]:

df_adm['ethnicity'] = df_adm['ethnicity'].str.upper().str.strip()
df_adm['ethnicity'] = df_adm['ethnicity'].replace({
    'WHITE': 'WHITE',
    'WHITE - OTHER EUROPEAN': 'WHITE',
    'WHITE - EASTERN EUROPEAN': 'WHITE',
    'WHITE - BRAZILIAN': 'WHITE',
    'WHITE - RUSSIAN': 'WHITE',
    'BLACK/AFRICAN AMERICAN': 'BLACK',
    'BLACK/CARIBBEAN ISLAND': 'BLACK',
    'ASIAN': 'ASIAN',
    'HISPANIC OR LATINO': 'HISPANIC',
    'HISPANIC/LATINO - PUERTO RICAN': 'HISPANIC',
    'UNKNOWN/NOT SPECIFIED': 'UNKNOWN',
    'UNABLE TO OBTAIN': 'UNKNOWN'
})

In [14]:
# correct mairtal status
df_adm['marital_status'] = df_adm['marital_status'].str.upper().str.strip()
df_adm['marital_status'] = df_adm['marital_status'].replace({
    'MARRIED': 'MARRIED',
    'SINGLE': 'SINGLE',
    'DIVORCED': 'DIVORCED',
    'WIDOWED': 'WIDOWED',
    'SEPARATED': 'SEPARATED',
    'UNKNOWN (DEFAULT)': 'UNKNOWN',
    'LIFE PARTNER': 'PARTNER'
})

In [15]:
#calc how long patients stayed at hosiptal?
df_adm['los_days_int'] = (df_adm['dischtime'] - df_adm['admittime']).dt.days
## Determine whether the death occurred during this stay.
df_adm['died_in_hospital'] = df_adm['deathtime'].notna()

In [16]:
df_adm.head(3)

,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,...,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data,los_days_int,died_in_hospital
0,12258,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,...,CATHOLIC,SEPARATED,BLACK,2164-10-23 16:43:00,2164-10-23 23:00:00,SEPSIS,False,True,8,False
1,12263,10011,105331,2126-08-14 22:32:00,2126-08-28 18:59:00,2126-08-28 18:59:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Private,...,CATHOLIC,SINGLE,UNKNOWN,NaT,NaT,HEPATITIS B,True,True,13,True
2,12265,10013,165520,2125-10-04 23:36:00,2125-10-07 15:13:00,2125-10-07 15:13:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,...,CATHOLIC,UNKNOWN,UNKNOWN,NaT,NaT,SEPSIS,True,True,2,True


In [19]:
# just extract date
df_adm['admit_date'] = df_adm['admittime'].dt.date
df_adm['discharge_date'] = df_adm['dischtime'].dt.date

In [20]:
df_adm['diagnosis'] = df_adm['diagnosis'].str.upper().str.strip()


df_adm['diagnosis'] = df_adm['diagnosis'].str.replace(r'[^\w\s]', ' ', regex=True)
df_adm['diagnosis'] = df_adm['diagnosis'].str.replace(r'\s+', ' ', regex=True)


df_adm['diagnosis'] = df_adm['diagnosis'].replace({
    'SEPSIS;PNEUMONIA;TELEMETRY': 'SEPSIS, PNEUMONIA',
    'STROKE/TIA': 'STROKE OR TIA',
    'HYPOTENSION;TELEMETRY': 'HYPOTENSION',
    'FEVER;URINARY TRACT INFECTION': 'FEVER, URINARY TRACT INFECTION'
})

In [22]:
print(inconsistent_deaths.to_string())

    row_id  subject_id  hadm_id           admittime           dischtime           deathtime admission_type    admission_location discharge_location insurance language       religion marital_status ethnicity           edregtime           edouttime       diagnosis  hospital_expire_flag  has_chartevents_data  los_days_int  died_in_hospital  admit_date discharge_date
72   39980       40687   129273 2155-03-08 02:35:00 2155-03-11 15:00:00 2155-03-12 15:00:00      EMERGENCY  EMERGENCY ROOM ADMIT       DEAD/EXPIRED  Medicare     ENGL  NOT SPECIFIED        WIDOWED     WHITE 2155-03-07 18:24:00 2155-03-08 03:45:00  ABDOMINAL PAIN                  True                  True             3              True  2155-03-08     2155-03-11


In [23]:
print(inconsistent_deaths[['subject_id', 'hadm_id', 'admittime', 'dischtime', 'deathtime']].to_string())


    subject_id  hadm_id           admittime           dischtime           deathtime
72       40687   129273 2155-03-08 02:35:00 2155-03-11 15:00:00 2155-03-12 15:00:00


In [24]:
df_adm.head(2)

,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,...,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data,los_days_int,died_in_hospital,admit_date,discharge_date
0,12258,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,...,BLACK,2164-10-23 16:43:00,2164-10-23 23:00:00,SEPSIS,False,True,8,False,2164-10-23,2164-11-01
1,12263,10011,105331,2126-08-14 22:32:00,2126-08-28 18:59:00,2126-08-28 18:59:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Private,...,UNKNOWN,NaT,NaT,HEPATITIS B,True,True,13,True,2126-08-14,2126-08-28


In [25]:
df_adm['marital_status'] = df_adm['marital_status'].astype('category')
df_adm['ethnicity'] = df_adm['ethnicity'].astype('category')


df_adm['admit_date'] = pd.to_datetime(df_adm['admit_date'], errors='coerce')
df_adm['discharge_date'] = pd.to_datetime(df_adm['discharge_date'], errors='coerce')



In [28]:
df_adm.to_csv("C:/Users/user/Downloads/BigData/dataset/mimic-iii-clinical-database-demo-1.4/cleaned_data/addmission.csv", index=False)

In [26]:
df_adm.to_parquet('D:/ITI Data engineering intensive camp/Big data/Project/Cleaned_data/ADMISSIONS.parquet', index=False)


In [38]:
df = df_adm.drop('los_days', axis=1)

In [26]:
df_adm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   row_id                129 non-null    int64         
 1   subject_id            129 non-null    int64         
 2   hadm_id               129 non-null    int64         
 3   admittime             129 non-null    datetime64[ns]
 4   dischtime             129 non-null    datetime64[ns]
 5   deathtime             40 non-null     datetime64[ns]
 6   admission_type        129 non-null    category      
 7   admission_location    129 non-null    category      
 8   discharge_location    129 non-null    category      
 9   insurance             129 non-null    category      
 10  language              129 non-null    category      
 11  religion              129 non-null    category      
 12  marital_status        129 non-null    category      
 13  ethnicity           

In [27]:
df_adm.isnull().sum()


row_id                   0
subject_id               0
hadm_id                  0
admittime                0
dischtime                0
deathtime               89
admission_type           0
admission_location       0
discharge_location       0
insurance                0
language                 0
religion                 0
marital_status           0
ethnicity                0
edregtime               37
edouttime               37
diagnosis                0
hospital_expire_flag     0
has_chartevents_data     0
los_days_int             0
died_in_hospital         0
admit_date               0
discharge_date           0
dtype: int64